In [1]:
import os
from config_exec import get_properties

EXECUTABLE: str = "py_executable.py"
EXTENSION: str = ".jv"
CONFIG_FILE: str = ""

PROPERTIES_OFFERED: list[str] = ["background-color", "width", "height", "title"]
LABEL_PROPERTIES_OFFERED: list[str] = ["color", "font-size", "background-color", "text-align", "font-family", "position"]

WINDOW_WIDTH: float = 1 # Used for position
WINDOW_HEIGHT: float = 1 # Used for position


def get_parse_file() -> list:
    """
    :return: list of .jv files
    """
    return [k for k in os.listdir() if k.endswith(EXTENSION)]


def get_parse_code(file_name: str) -> list:
    with open(file_name, "r") as fs:
        return fs.read().split("\n")


def write_to_executable(command: str) -> None:
    with open(EXECUTABLE, "a") as fs:
        fs.write(command + "\n")
        
def handle_properties(properties: dict) -> None:
    global WINDOW_WIDTH, WINDOW_HEIGHT
    property_actions = {
        "background-color": lambda name, value: f"{name}.config(bg='{value}')",
        "width": lambda name, value: f"window.geometry('{value}x800')",
        "height": lambda name, value: f"window.geometry('800x{value}')",
        "title": lambda name, value: f"window.title('{value}')",
        "font-size": lambda name, value: f"{name}.config(font=('', {value.split('px')[0]}))",
        "text-align": lambda name, value: f"{name}.config(anchor='{ {'left': 'w', 'center': 'center', 'right': 'e'}.get(value.lower(), 'w') }')",
        "font-family": lambda name, value: f"{name}.config(font=('', 12, '{value}'))",
        "color": lambda name, value: f"{name}.config(fg='{value}')",
        "position": lambda name, value: f"{name}.place(x={int(value.split(',')[0].strip()) / 100 * WINDOW_HEIGHT}, y={int(value.split(',')[1].strip()) / 100 * WINDOW_WIDTH})",
    }

    if 'window' in properties:
        window_props = properties['window']
        for prop in PROPERTIES_OFFERED:
            if prop in window_props:
                value = window_props[prop]
                write_to_executable(property_actions[prop]("window", value))
                
                if prop.strip() == "width":
                    WINDOW_WIDTH = float(value)
                elif prop.strip() == "height":
                    WINDOW_HEIGHT = float(value)
                    
        del properties['window']

    for variable_name, attributes in properties.items():
        for prop, value in attributes.items():
            if prop in LABEL_PROPERTIES_OFFERED:
                write_to_executable(property_actions[prop](variable_name, value))



def println(command: str) -> str:
    to_print: str = command[command.index("->") + 2:].strip()
    variable_name: str = to_print[to_print.index("id="):].strip().split("id=")[1][1:-1]
    text_to_be_printed: str = to_print[:to_print.index("id=")].strip()[:-1]
    command_string: str = f"{variable_name} = Label(window, text={text_to_be_printed}, fg='black')\n{variable_name}.place(x=4, y=4)\n"
    return command_string


def unpack_command(command: str) -> str:
    if "//" in command:
        command = command[:command.index("//")]
        
    space_less: str = command.replace(" ", "")
        
    if "!println->" in space_less and space_less.index("!println") == 0:
        return println(command)
    elif "#CONFIG" in space_less:
        global CONFIG_FILE
        CONFIG_FILE = space_less.split(" ", 1)[1]
    else:
        raise Exception(f"Command not recognized: {command}")


def get_defaults(parent_type: str) -> dict:
    defaults_dict_window = {
        "background-color": "white",
        "color": "black",
        "width": 800,
        "height": 800,
        "title": "javelinLang"
    }

    if parent_type == "window":
        return defaults_dict_window


def get_all_properties() -> dict:
    properties: dict = get_properties(CONFIG_FILE)
    
    all_properties_for_this_element: dict = {}

    if 'window' in properties:
        window_properties = properties['window']
        window_props: dict = {}
        for this_property in PROPERTIES_OFFERED:
            if this_property in window_properties:
                window_props[this_property] = window_properties[this_property]
            else:
                window_props[this_property] = get_defaults("window")[this_property]
        all_properties_for_this_element['window'] = window_props
        del properties['window']
    
    for variable_list, attributes in properties.items():
        variable_list: list = variable_list.split(",")  # Split the variable list by comma
        for variable_name in variable_list:
            all_properties_for_this_element[variable_name.strip()] = attributes
            
    return all_properties_for_this_element


def master(file_data: list) -> None:
    global CONFIG_FILE
    for command in [line for line in file_data if line.strip() != ""]:
        if "#CONFIG" in command.strip():
            CONFIG_FILE = command.strip().split(" ", 1)[1]
        
        if command.strip()[0] != "!":
            write_to_executable(command)
        else:
            unpacked_command: str = unpack_command(command.strip())
            write_to_executable(unpacked_command)

    handle_properties(get_all_properties())
    write_to_executable("\nwindow.mainloop()")



def read_config() -> str:
    with open(CONFIG_FILE, "r") as fs:
        return fs.read()


def main() -> None:
    # Write initial setup to py_executable.py
    with open(EXECUTABLE, "w") as fs:
        fs.write("from tkinter import *\n\nwindow = Tk()\n")

    # Process .jv files
    for file_name in get_parse_file():
        file_data = get_parse_code(file_name)
        master(file_data)

    exec(open("py_executable.py").read())


if __name__ == "__main__":
    main()